In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import numpy as np
import pandas as pd
import sklearn
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from sklearn import feature_extraction,model_selection
from sklearn.metrics import confusion_matrix
import glob
import os
import string
import tensorflow as tf
import keras as K
import zipfile
from itertools import chain
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import matplotlib.pyplot as plt
from sklearn.metrics import plot_confusion_matrix, classification_report

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Using TensorFlow backend.


In [0]:
# importing Word2Vec
from gensim.models import Word2Vec

In [0]:
zip_ref = zipfile.ZipFile("/content/drive/My Drive/Colab Notebooks/hate-speech-dataset-master.zip", 'r')
zip_ref.extractall()
zip_ref.close()
file_path="/content/hate-speech-dataset-master/all_files"

In [6]:
read_files=glob.glob(os.path.join(file_path,"*.txt"))
data=[]
file_name=[]
for files in read_files:
    with open(files,encoding='utf-8') as f:
        data.append(f.read())
        file_name.append(os.path.splitext(os.path.basename(files))[0])
data=pd.DataFrame(data,index=file_name)
data.head()

0
13639258_1  I just cannot understand someone wanting to ri...
30680004_3  What educational goals or philosophies influen...
13504039_3  Los Angeles County sheriff 's officials arrest...
13465908_1  France has been gone for a long time , we will...
14665319_1                                     84% French ...

In [7]:
Output=pd.read_csv("/content/hate-speech-dataset-master/annotations_metadata.csv",index_col=0)
Output=Output.drop(['user_id','subforum_id','num_contexts'],axis=1).sort_index()
Output.head()

label
file_id            
12834217_1   noHate
12834217_10  noHate
12834217_2   noHate
12834217_3   noHate
12834217_4     hate

In [8]:
dataSet=pd.merge(data,Output,how='outer',left_index=True,right_index=True)
dataSet=dataSet[(dataSet['label']=="noHate")|(dataSet['label']=='hate')]
dataSet.rename(columns={0:'Texts'},inplace=True)
dataSet['label']=(dataSet['label']=='hate').astype(int)
dataSet.head()

Texts  label
12834217_1   As of March 13th , 2014 , the booklet had been...      0
12834217_10  Thank you in advance. : ) Download the youtube...      0
12834217_2   In order to help increase the booklets downloa...      0
12834217_3   ( Simply copy and paste the following text int...      0
12834217_4   Click below for a FREE download of a colorfull...      1

In [9]:
x = dataSet['Texts']
y = dataSet['label']
print(x.head(), y.head())

12834217_1     As of March 13th , 2014 , the booklet had been...
12834217_10    Thank you in advance. : ) Download the youtube...
12834217_2     In order to help increase the booklets downloa...
12834217_3     ( Simply copy and paste the following text int...
12834217_4     Click below for a FREE download of a colorfull...
Name: Texts, dtype: object 12834217_1     0
12834217_10    0
12834217_2     0
12834217_3     0
12834217_4     1
Name: label, dtype: int64


In [10]:
from keras.utils.np_utils import to_categorical
y = to_categorical(np.asarray(y))
print(x)

12834217_1     As of March 13th , 2014 , the booklet had been...
12834217_10    Thank you in advance. : ) Download the youtube...
12834217_2     In order to help increase the booklets downloa...
12834217_3     ( Simply copy and paste the following text int...
12834217_4     Click below for a FREE download of a colorfull...
                                     ...                        
33677015_1     Apparently he came to the conclusion that his ...
33677019_1     Wish we at least had a Marine Le Pen to vote f...
33677019_2     Its like the choices are white genocide candid...
33677053_1     Why White people used to say that sex was a si...
33677053_2                                        Now I get it !
Name: Texts, Length: 10703, dtype: object


In [0]:
tk=Tokenizer(filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',lower=True, split=" ")
tk.fit_on_texts(x)
word_index=tk.word_index
#print(index)
x2 = tk.texts_to_sequences(x)

In [13]:
embedding_vecor_length =146
padded_docs = pad_sequences(x2, maxlen=embedding_vecor_length, padding='post')
print (padded_docs)

[[ 26   6 515 ...   0   0   0]
 [200  11   7 ...   0   0   0]
 [  7 701   2 ...   0   0   0]
 ...
 [111  28   1 ...   0   0   0]
 [148  18  43 ...   0   0   0]
 [100   3  53 ...   0   0   0]]


In [14]:
vocab_size = len(tk.word_index)
print(vocab_size)

16002


In [18]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(padded_docs, y, test_size=0.30, random_state=42)
!pip install --upgrade gensim

     |████████████████████████████████| 24.2MB 1.2MB/s 
  Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


In [38]:
from gensim.models import Word2Vec,KeyedVectors
import pandas as pd
mode=KeyedVectors.load_word2vec_format("/content/drive/My Drive/Colab Notebooks/GoogleNews-vectors-negative300.bin.gz",binary=True,limit=1000000)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


Total 400000 word vectors in Glove 6B 100d.


In [39]:
# Initialize the weight for each word for the embedding layer. Words not present
# in the dictionary initialized by random value 
EMBEDDING_DIM = 300
embedding_matrix = np.random.random((len(word_index) + 1, EMBEDDING_DIM))
#embedding_matrix = (16433, 100) NOTE: It is from uniques words in our tweets
for word, i in word_index.items(): # Selecting word from our tweet unique words list for changing with the values from glove
    if word in mode.wv.vocab:
      embedding_vector =mode[word] # Finding our tweet words in glove and taking its array of 100 values
      embedding_matrix[i] = embedding_vector

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """


In [40]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalMaxPooling1D, MaxPooling1D
from keras.utils.vis_utils import plot_model
model = Sequential()
model.add(Embedding(input_dim = vocab_size + 1, output_dim = 300, weights=[embedding_matrix], input_length=embedding_vecor_length))
model.add(Conv1D(filters = 128,kernel_size=3, activation='relu'))
model.add(MaxPooling1D(5))
model.add(Flatten())
model.add(Dense(256, activation = 'relu'))
model.add(Dense(2, activation = 'softmax'))
model.compile(loss='binary_crossentropy', optimizer='Adam', metrics=['accuracy'])
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 146, 300)          4800900   
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 144, 128)          115328    
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 28, 128)           0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 3584)              0         
_________________________________________________________________
dense_5 (Dense)              (None, 256)               917760    
_________________________________________________________________
dense_6 (Dense)              (None, 2)                 514       
Total params: 5,834,502
Trainable params: 5,834,502
Non-trainable params: 0
____________________________________________

In [46]:
model.fit(X_train, y_train, validation_data = (X_test, y_test),epochs=100,batch_size=100, verbose=2)

Train on 7492 samples, validate on 3211 samples
Epoch 1/100
 - 1s - loss: 2.7788e-05 - acc: 1.0000 - val_loss: 0.7024 - val_acc: 0.8944
Epoch 2/100
 - 1s - loss: 2.5635e-05 - acc: 1.0000 - val_loss: 0.7083 - val_acc: 0.8944
Epoch 3/100
 - 1s - loss: 2.3751e-05 - acc: 1.0000 - val_loss: 0.7145 - val_acc: 0.8947
Epoch 4/100
 - 1s - loss: 2.2107e-05 - acc: 1.0000 - val_loss: 0.7182 - val_acc: 0.8947
Epoch 5/100
 - 1s - loss: 2.0502e-05 - acc: 1.0000 - val_loss: 0.7232 - val_acc: 0.8944
Epoch 6/100
 - 1s - loss: 1.9100e-05 - acc: 1.0000 - val_loss: 0.7247 - val_acc: 0.8938
Epoch 7/100
 - 1s - loss: 1.7764e-05 - acc: 1.0000 - val_loss: 0.7312 - val_acc: 0.8944
Epoch 8/100
 - 1s - loss: 1.6599e-05 - acc: 1.0000 - val_loss: 0.7359 - val_acc: 0.8944
Epoch 9/100
 - 1s - loss: 1.5536e-05 - acc: 1.0000 - val_loss: 0.7380 - val_acc: 0.8938
Epoch 10/100
 - 1s - loss: 1.4515e-05 - acc: 1.0000 - val_loss: 0.7396 - val_acc: 0.8932
Epoch 11/100
 - 1s - loss: 1.3628e-05 - acc: 1.0000 - val_loss: 0.7413 

In [47]:
predictions_test = model.predict(X_test)
print(predictions_test)

[[1.0000000e+00 4.9443956e-12]
 [1.0000000e+00 9.4792283e-16]
 [1.0000000e+00 3.4723597e-16]
 ...
 [1.0000000e+00 8.3609606e-15]
 [1.0000000e+00 1.3010446e-09]
 [1.0000000e+00 1.5788408e-09]]


In [49]:
X_test

array([[  149,    11,    14, ...,     0,     0,     0],
       [   97,    68, 10811, ...,     0,     0,     0],
       [    1,  1139,   825, ...,     0,     0,     0],
       ...,
       [   25,   111,   650, ...,     0,     0,     0],
       [    3,  1100,   117, ...,     0,     0,     0],
       [   17,     8,  2595, ...,     0,     0,     0]], dtype=int32)

In [48]:
predictions_test1 = np.zeros_like(predictions_test)
predictions_test1[np.arange(len(predictions_test)), predictions_test.argmax(1)] = 1
predictions_test1

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]], dtype=float32)

In [44]:
print(classification_report(y_test,predictions_test1))

              precision    recall  f1-score   support

           0       0.92      0.96      0.94      2842
           1       0.56      0.38      0.45       369

   micro avg       0.89      0.89      0.89      3211
   macro avg       0.74      0.67      0.70      3211
weighted avg       0.88      0.89      0.89      3211
 samples avg       0.89      0.89      0.89      3211



In [45]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test,predictions_test1))

0.8947368421052632
